### Demo: insepct DataClass properties
*(built during problem scoring loaded test data bug)*


In [1]:
from fastai2.vision.all import *

def piece_class_parse(fn): 
    fn = fn.split('_')[1]
    fn = fn.split('.')[0]
    return fn

path = Path('../../../rf-chess-data/cropped_v1/')

In [2]:
pieces = ImageDataLoaders.from_name_func(
                path, 
                get_image_files(path),
                valid_pct=0.2, 
                seed=42,
                label_func=piece_class_parse, 
                item_tfms=RandomResizedCrop(128, min_scale=0.5),
                batch_tfms=aug_transforms(),
                )

In [8]:
pieces2 = ImageDataLoaders.from_name_func(
                path, 
                get_image_files(path),
                valid_pct=0.2, 
                seed=42,
                label_func=piece_class_parse, 
                item_tfms=RandomResizedCrop(128, min_scale=0.5),
                batch_tfms=aug_transforms(),
                )

##### Build on it

In [53]:
pieces.train.get_idxs()[:4]

[1657, 1554, 531, 191]

In [47]:
pieces.valid.get_idxs()[:4]

[0, 1, 2, 3]

In [55]:
pieces.valid.items[:3]

[Path('../../../rf-chess-data/cropped_v1/00142_black-pawn.jpg'),
 Path('../../../rf-chess-data/cropped_v1/02374_white-king.jpg'),
 Path('../../../rf-chess-data/cropped_v1/01859_black-king.jpg')]

In [98]:
t_tfm = learn.dls[0]

In [99]:
t_tfm.tfms

(#2) [Pipeline: PILBase.create,Pipeline: partial -> Categorize]

In [100]:
v_tfm = learn.dls[1]

In [101]:
v_tfm.tfms

(#2) [Pipeline: PILBase.create,Pipeline: partial -> Categorize]

In [107]:
[e for e in dir(t_tfm) if 'item' in e]

['_after_item',
 'after_item',
 'after_item',
 'create_item',
 'do_item',
 'itemgot',
 'items',
 'items']

##### by default, a training dl is shuffled and the valid is not

In [139]:
pieces.train.shuffle, pieces.valid.shuffle

(True, False)

##### the shuffling is seen in `do_item()` which will pull diff images for same index only in train but the same in valid

In [123]:
t_tfm.do_item(0)[0][0][0][0], t_tfm.do_item(0)[0][0][0][0]

(tensor(30, dtype=torch.uint8), tensor(1, dtype=torch.uint8))

In [126]:
v_tfm.do_item(0)[0][0][0][0], v_tfm.do_item(0)[0][0][0][0]

(tensor(222, dtype=torch.uint8), tensor(222, dtype=torch.uint8))

##### `do_item()` on each of the two `dls` returns the same image
implying that transforms are off or the same

In [138]:
pieces.valid.do_item(0)[0].sum(), pieces2.valid.do_item(0)[0].sum()

(TensorImage(3383588), TensorImage(3383588))

##### Using pipelines

In [81]:
pp_y = v.tfms[1]
pp_y.show(1)

black-king


In [82]:
pp_x = v.tfms[0]
pp_x.show(0)